### Resource

- [google drive](https://drive.google.com/drive/u/1/folders/1N15PTA1y0ou5YyPDUN-MdlTGPXq3Rxhc)
- [iowa liquor dataset](https://console.cloud.google.com/bigquery?authuser=1&invt=AbtrQg&project=solid-dominion-452916-p4&supportedpurview=project&ws=!1m5!1m4!4m3!1sbigquery-public-data!2siowa_liquor_sales!3ssales)
- [iowa population by county](https://data.iowa.gov/Community-Demographics/County-Population-in-Iowa-by-Year/qtnr-zsrc/about_data)
- [iowa population by city](https://data.iowa.gov/Community-Demographics/Iowa-Population-by-City-ACS-5-Year-Estimates-/26ad-icvy/about_data)
- [iowa income](https://data.iowa.gov/Economic-Statistics/Annual-Personal-Income-for-State-of-Iowa-by-County/st2k-2ti2/about_data)
- [iowa fuel sales](https://data.iowa.gov/Sales-Distribution/Iowa-Motor-Fuel-Sales-by-County-and-Year/hbwp-wys3/about_data)
- [iowa excessive drinking](https://www.countyhealthrankings.org/health-data/community-conditions/health-infrastructure/health-promotion-and-harm-reduction/excessive-drinking?state=19&tab=1&year=2025)

### Packages

In [2]:
import pandas as pd
import re

# 1. Data Frame

In [13]:
df_store = pd.read_csv('data/google.csv')
df_store = (
    df_store
    .groupby(['store', 'city', 'county', 'date', 'liter', 'liquor_type'])
    .agg({
        'gross_profit': 'sum',
    })
    .reset_index()
)
df_store.to_csv('data/store.csv', index=False)

# categorization
df_store['month'] = pd.to_datetime(df_store['date']).dt.month
df_store = (
    df_store.groupby(['store', 'month'])
    .agg({
        'city': 'first', 
        'county': 'first',
        'liter': 'sum',
        'gross_profit': 'sum'
    })
    .reset_index()
)

def assign_category(store_name):
    store = store_name.lower() if isinstance(store_name, str) else ""
    store_clean = store
    categories = {
        "grocery_store": ["groc", "super saver iv", "sunshine foods", "sioux food center", "shivam food mart llc", "save more", "food pride", "sam's food", "sam food mart", "roy's foodland", "quillins quality foods", "quillins food ranch", "osco #1118", "state food mart", "strawberry food", "stratford food center", "super saver iv," "sam food mart", "ruback's food center", "terry's food center", "sam's club", "target", "price chopper", "quillins quality foods west union", "r&l foods", "mepo foods", "montezuma super valu", "mcnally's super valu", "maynards", "mapleton food land", "logan super foods", "laurens food pride", "lake city food center", "lansing iga", "lake view foods",  "la tropicana", "kyaw family, inc", "jim and charlies affiliated", "keota eagle foods", "jubilee foods", "jeff's foods", "jeffs foods", "jack & jill store", "ida grove food pride", "gary's foods", "grandview mart", "hy-vee", "hometown foods", "the food center", "frohlich's super valu", "fredricksburg food center", "freeman foods", "fire and salt co", "forbes meat", "feista foods", "family fare",  "fairbank food center", "el valle tienda y carniceria", "davids foods", "thriftway", "country foods", "tipton family foods", "center point foods", "dashmart", "corwith farm service", "clear lake payless foods", "britt food center", "fareway stores", "dutchmart", "eagle food mart", "bob & scott's", "cash saver", "best food mart", "byron mart", "big g food store", "bender's foods", "bender foods", "trunck's country foods, inc", "avoca food land", "avenue g store", "audubon food land", "costco", "whole foods", "trader joe", "wmart", "fresh", "wilkes", "wikes", "market", "wal-mart"],
        "liquor_store_bar": ["8 seconds", "central city 2", "riverside #4 / bettendorf", "sun mart", "subedi llc", "ss khanna group inc", "south side food mart", "smokin hot", "sip & burn", "sid's beverage shop", "royal mart", "puff pour play inc", "jumbo's", "super quick", "the hut", "prime mart", "swarm sports bar and grille", "oasis", "the cooler", "the beverage shop", "new star", "new hampton mart", "metro mart", "malik's",  "local craft cellar", "last call 2", "kimberly mart", "hurribak iv distribution", "great pastimes", "grieder beverage depot", "the secret cellar", "essentials plus", "essentials",  "eldridge mart", "easygo", "downtown pantry", "division mart", "ding's honk'n holler", "concordia llc", "chuck's sportsmans beverage", "bani's", "cork 'n bottle", "cork and bottle", "cork it!", "brady mart", "benz distributing", "brewski", "blind pig cocktail bodega", "bootleg", "brew ida grove", "camanche food pride", "beer", "another round", "ambysure inc", "whiskey", "shack", "liquor", "booze", "spirits", "wine", "smoke", "tobacco", "ali's"],
        "gas_station": ["yana mart", "super convenience store", "speede shop", "southgate ames", "snak atak #53", "site food mart", "sioux-per center", "sinclair", "select mart", "rolfe heartland", "river mart", "rina mart llc", "quik and handi", "quick shop foods", "quick mart", "quick corner", "pump n pack", "primestar", "porters convenience 66", "pony express", "oak street station llc", "speedee mart", "stan's corner", "star / denison", "star convenience", "super mart", "quik n handi iii", "truck plaza inc.", "shop n save", "pump 'n pak", "raymond's station", "taylor's", "phillips 66", "the spot #3", "otho convenience and food", "oskaloosa watering hole", "oly's c store", "oelwein mart", "panther travel center", "quik pik", "raceway 80", "quik trip", "the pumper", "the corner store", "the boonedocks", "naaz group llc", "moes mart", "mod /", "mk minimart, inc", "mini-mart", "mega saver", "mega retail", "mcdermott oil", "mason mart", "martelle store", "locust mart", "lefty's convenience", "l&m mighty shop", "kramer's inc.", "jiffy", "the depot", "kwik star", "kwik shop", "konvenience korner", "kline's quick time", "kimmes", "kj's convenience & deli", "karam kaur khasriya llc", "jj's valley mart", "country store", "indy 66", "gd xpress", "garner foods", "fastland llc", "crossroads / hampton", "the depot williamsburg", "hill country convenience store", "hill brothers jiffy mart", "hello mart llc", "golden mart", "good and quick co", "hawkeye convenience store", "filling station", "the station", "speed shoppe", "speede shoppe", "fast break", "ampride", "fas mart", "engen express", "ehlinger's vinton express", "dhakals llc", "dewitt travel mart", "depot norway", "decorah mart", "day break", "cresco mart", "corydon corner", "circle s", "budzz", "ez mart", "e z mart", "express mart", "cubby's", "corner mart", "bucky's", "central mart i, llc", "conoco", "car-go-express", "circle k", "best trip", "thunder ridge", "the station ii", "calamus country store", "brew floyd", "tiger mart", "casey's", "brew oil", "shell", "bailey's convenience", "b and b west", "b p on 1st", "uni mart", "andrew country mart", "westland fast", "westside houser mart", "junction", "gas", "sheetz", "four corners", "petro", "wawa", "bp", "quiktrip", "buc-ee", "380bp", "fuel", "kum", "stop", "yesway", "wilton express", "williams travel"],
        "pharmacy": ["pharmacy", "cvs", "walgreen", "cornerstone apothecary", "hartig drug"],
        "distillery_brewery": ["distill", "legendary rye", "brewery", "vineyard", "catfish charlie's"],
        "general_store": ["general", "bellevue express", "guppy's on the go", "huber's store", "gateway mart" ],
        "convenience_store": ["7/11", "sahota food mart", "station mart", "sycamore convenience", "mughal, inc", "neighborhood mart", "moti's food", "mad ave quik shop", "kc brothers", "jim's food", "east village pantry", "family pantry", "gm minimart", "gm mart", "gm food mart",  "bravim mart incorporated", "carson country store llc", "big 10 mart", "umiya foodmart inc", "7 days mart", "valley west corner store", "mini mart", "west k mart"],
        "other": ["williams boulevard service", "sodes green acre", "prairie meadows", "gopuff", "ib pony", "new york dollar store", "franklin street floral & gift", "eichman enterprises inc", "inn & suites", "budget inn", "10th", "adventureland", "whiting", "casino", "w&h cooperative oil co", "troy mills / troy mills" ],
        "unknown" : ["best deals / waterloo", "k-zar inc - waterloo", "kc store / stratford", "larchwood offsale", "rienbeck gp llc / reinbeck"] 
    }
    for category, keywords in categories.items():
        if any(keyword in store_clean for keyword in keywords):
            return category
    return "uncategorized"
df_store['category'] = df_store['store'].apply(assign_category)
df_store = pd.get_dummies(df_store, columns=['category'], prefix='s')

# county
df_store = pd.get_dummies(df_store, columns=['county'], prefix='c')

# trasform bool to int
bool_cols = df_store.select_dtypes(include='bool').columns
df_store[bool_cols] = df_store[bool_cols].astype(int)

df_store

,store,month,city,liter,gross_profit,s_convenience_store,s_distillery_brewery,s_gas_station,s_general_store,s_grocery_store,...,c_wapello,c_warren,c_washington,c_wayne,c_webster,c_winnebago,c_winneshiek,c_woodbury,c_worth,c_wright
0,'da booze barn / west bend,1,west bend,81.300,4499.84,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,'da booze barn / west bend,2,west bend,80.325,4598.18,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,'da booze barn / west bend,3,west bend,73.475,4840.74,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,'da booze barn / west bend,4,west bend,68.100,3828.55,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,'da booze barn / west bend,5,west bend,87.750,4691.36,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23090,yesway store #1198 / swea city,7,swea city,19.925,1773.04,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
23091,yesway store #1198 / swea city,9,swea city,8.250,626.34,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
23092,yesway store #1198 / swea city,10,swea city,11.375,653.52,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
23093,yesway store #1198 / swea city,11,swea city,8.000,642.18,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
# widen liquor type
df_liquor_type = pd.read_csv('data/store.csv')
df_liquor_type['month'] = pd.to_datetime(df_liquor_type['date']).dt.month
df_liquor_type = df_liquor_type.pivot_table(
    index=['store', 'month'],   
    columns='liquor_type',            
    values='liter',          
    aggfunc='sum',              
    fill_value=0            
)
df_liquor_type = df_liquor_type.reset_index()
def clean_column_name(col):
    col = str(col).lower()
    col = col.replace('/', '_')  
    col = col.replace('*', '')
    col = col.replace('.', '_')  
    col = col.replace(':', '')
    col = col.replace('(', '')
    col = col.replace(')', '')
    col = col.replace(',', '')
    col = re.sub(r'\s+', '_', col)  
    col = re.sub(r'_+', '_', col)  
    col = re.sub(r'[^a-z0-9_]', '_', col)   
    col = re.sub(r'_+', '_', col)          
    col = col.strip('_')       
    return 'l_' + col             
df_liquor_type.columns = [clean_column_name(col) for col in df_liquor_type.columns]
df_liquor_type.rename(columns={
    "l_store": "store",
    "l_month": "month"
}, inplace=True)

# merge 
df_algorithm = pd.merge(df_store, df_liquor_type, on=['store', 'month'], how='inner')
df_algorithm = df_algorithm.drop_duplicates()
df_algorithm.to_csv('data/iowa_algorithm.csv', index=False)
df_algorithm

,store,month,city,liter,gross_profit,s_convenience_store,s_distillery_brewery,s_gas_station,s_general_store,s_grocery_store,...,l_single_malt_scotch,l_special_order_items,l_spiced_rum,l_straight_bourbon_whiskies,l_straight_rye_whiskies,l_temporary_specialty_packages,l_tennessee_whiskies,l_triple_sec,l_whiskey_liqueur,l_white_rum
0,'da booze barn / west bend,1,west bend,81.300,4499.84,0,0,0,0,0,...,0.0,0.00,4.300,8.150,0.00,5.50,5.425,0.0,2.250,1.75
1,'da booze barn / west bend,2,west bend,80.325,4598.18,0,0,0,0,0,...,1.5,0.00,5.250,9.000,0.00,2.50,0.750,0.0,2.500,0.00
2,'da booze barn / west bend,3,west bend,73.475,4840.74,0,0,0,0,0,...,0.0,0.00,5.250,5.750,0.75,2.50,1.750,1.0,4.250,0.00
3,'da booze barn / west bend,4,west bend,68.100,3828.55,0,0,0,0,0,...,0.0,0.00,5.250,6.000,1.00,2.50,1.750,0.0,0.750,1.75
4,'da booze barn / west bend,5,west bend,87.750,4691.36,0,0,0,0,0,...,0.0,1.75,2.800,8.625,0.75,4.00,4.050,1.0,1.750,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23090,yesway store #1198 / swea city,7,swea city,19.925,1773.04,0,0,1,0,0,...,0.0,0.00,2.875,0.050,0.00,0.00,0.000,0.0,5.775,0.00
23091,yesway store #1198 / swea city,9,swea city,8.250,626.34,0,0,1,0,0,...,0.0,0.00,0.000,0.000,0.00,0.00,0.000,0.0,1.800,0.00
23092,yesway store #1198 / swea city,10,swea city,11.375,653.52,0,0,1,0,0,...,0.0,0.00,0.000,0.000,0.00,0.75,0.000,0.0,0.000,0.00
23093,yesway store #1198 / swea city,11,swea city,8.000,642.18,0,0,1,0,0,...,0.0,0.00,0.375,0.750,0.00,0.00,0.000,0.0,0.000,0.00


# 2. Reference

In [19]:
# widen liquor type
df_ref = pd.read_csv('data/store.csv')
df_ref['month'] = pd.to_datetime(df_ref['date']).dt.month
df_ref = df_ref.pivot_table(
    index=['county', 'month'],   
    columns='liquor_type',            
    values='liter',          
    aggfunc=['mean', 'max'],            
    fill_value=0            
)
df_ref = df_ref.reset_index()
def clean_column_name(col):
    col = str(col).lower()
    col = col.replace('/', '_')  
    col = col.replace('*', '')
    col = col.replace('.', '_')  
    col = col.replace(':', '')
    col = col.replace('(', '')
    col = col.replace(')', '')
    col = col.replace(',', '')
    col = re.sub(r'\s+', '_', col)  
    col = re.sub(r'_+', '_', col)  
    col = re.sub(r'[^a-z0-9_]', '_', col)   
    col = re.sub(r'_+', '_', col)          
    col = col.strip('_')       
    return col             
df_ref.columns = [clean_column_name(col) for col in df_ref.columns]
df_ref = df_ref.round(2)
df_ref.to_csv('table/ref_iowa_county.csv', index=False)
df_ref

,county,month,mean_100_agave_tequila,mean_aged_dark_rum,mean_american_brandies,mean_american_cordials_liqueurs,mean_american_distilled_spirits_specialty,mean_american_dry_gins,mean_american_flavored_vodka,mean_american_schnapps,...,max_single_malt_scotch,max_special_order_items,max_spiced_rum,max_straight_bourbon_whiskies,max_straight_rye_whiskies,max_temporary_specialty_packages,max_tennessee_whiskies,max_triple_sec,max_whiskey_liqueur,max_white_rum
0,adair,1,0.54,0.00,1.04,0.52,0.75,0.00,0.61,0.20,...,0.00,0.0,1.75,1.75,0.00,0.05,0.75,0.00,1.75,1.75
1,adair,2,0.66,0.00,0.75,0.05,0.75,0.75,0.79,0.26,...,0.75,0.0,1.75,1.75,0.75,0.05,1.75,0.00,1.75,1.75
2,adair,3,0.54,0.00,1.15,0.28,0.00,0.38,0.74,0.27,...,0.00,0.0,1.75,1.75,0.00,0.75,0.75,0.75,1.75,1.75
3,adair,4,0.65,0.00,0.75,0.05,0.75,0.56,0.50,0.37,...,0.00,0.0,1.75,1.75,0.75,0.75,0.75,0.75,1.75,0.75
4,adair,5,0.58,0.75,0.91,0.23,0.75,1.06,0.51,0.29,...,0.00,0.0,1.75,1.75,0.75,0.75,1.75,1.00,1.75,1.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1183,wright,8,0.70,0.00,0.80,0.23,0.75,1.56,0.67,0.58,...,0.75,0.0,1.75,1.75,0.75,0.05,1.75,0.00,1.75,1.75
1184,wright,9,0.68,0.00,0.86,0.17,0.75,1.50,0.69,0.67,...,0.75,0.0,1.75,1.75,0.75,0.75,1.75,1.00,1.75,1.75
1185,wright,10,0.66,0.00,0.66,0.28,0.75,0.93,0.54,0.54,...,0.75,0.0,1.75,1.75,1.00,1.75,1.75,1.00,1.75,1.75
1186,wright,11,0.61,0.75,0.91,0.50,0.40,1.42,0.61,0.62,...,0.75,0.0,1.75,1.75,1.00,0.75,1.75,0.75,1.75,1.75
